# Predict Leaf Disease Notebook

This notebook runs the Python script to predict leaf diseases using a trained model.

In [ ]:
import argparse
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image, img_to_array, load_img

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser(
        description='Predict leaf disease from a single image using a trained model.')
    parser.add_argument('image_path', help='Path to the input image.')
    parser.add_argument('--model_zip', default='trained_model_and_augmented.zip',
                        help='Zip file containing the trained model.')
    parser.add_argument('--model_file', default='trained_leaf_disease_model.h5',
                        help='H5 model file inside the zip.')
    return parser.parse_args()

In [ ]:
def load_model_from_zip(zip_path, model_filename):
    """
    Extracts the .h5 file from the zip, loads it with Keras, then returns the model.
    """
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extract(model_filename, path='.')
    model = tf.keras.models.load_model(model_filename)
    return model

In [ ]:
def predict_image(model, img_path):
    """
    Preprocess the image, run model prediction, and return the predicted class and its confidence.
    """
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_idx = int(np.argmax(preds[0]))
    confidence = float(preds[0][class_idx])
    return class_idx, confidence

In [ ]:
def main():
    args = parse_args()
    
    if not os.path.isfile(args.image_path):
        print(f'Error: {args.image_path} is not a valid file.')
        return
    
    if not os.path.isfile(args.model_zip):
        print(f'Error: {args.model_zip} does not exist.')
        return
    
    model = load_model_from_zip(args.model_zip, args.model_file)
    print('Model loaded successfully.')
    
    class_idx, confidence = predict_image(model, args.image_path)
    
    class_names = [
        'apple_apple_scab',
        'apple_black_rot',
        'apple_cedar_apple_rust',
        'apple_healthy'
    ]
    
    predicted_class = class_names[class_idx] if class_idx < len(class_names) else 'Unknown'
    
    img_original = load_img(args.image_path)
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    axes[0].imshow(img_original)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    resized_img = load_img(args.image_path, target_size=(224, 224))
    axes[1].imshow(resized_img)
    axes[1].set_title(f'Transformed (Resized)\nPred: {predicted_class}\nConf: {confidence:.2f}')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
main()